In [1]:
import pandas as pd
import json
import spacy
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [3]:
sd_data = pd.read_csv('data/train_chat.csv')
sd_data_manual = pd.read_csv('data/speed-dating_preprocessed_manual.csv')
sd_data_pp = pd.read_csv('data/speed-dating_preprocessed.csv')

In [25]:
def extract_questions(dataset):
    q_list = []
    for i in range(len(dataset)):
      #text = metadata2['Message'][i] #speed-dating dataset
      text = dataset[i] #persona chat dataset
      text = text.replace('|', '. ')
      text_sentences = sent_tokenize(text)
      for sentence in text_sentences:
        tokens = word_tokenize(sentence)
        for token in sentence:
          if token == '?':
            q_list.append(sentence)
    return q_list


In [32]:
import spacy
nlp = spacy.load('en_core_web_sm')
def extract_noun(data):
    noun_dict = {}
    for i in range(len(data)):
      text = data[i] #persona chat dataset
      text = nlp(text)
      for token in text:
          #print(token.text, token.pos_, token.dep_)
          if token.pos_ == "NOUN":
            #TODO lemmatization
            if token.text not in noun_dict.keys():
              noun_dict[token.text] = 1
            else:
              noun_dict[token.text] +=1
    return noun_dict


In [26]:
q_1 =extract_questions(sd_data['Message'])
q_2 =extract_questions(sd_data_manual.iloc[ : , 0 ])
q_3 =extract_questions(sd_data_pp['0'])

In [34]:
nouns_sd = extract_noun(q_1)
nouns_sd_m = extract_noun(q_2)
nouns_sd_pp = extract_noun(q_3)

In [40]:
import operator
noun_dict_sd = sorted(nouns_sd.items(), key=operator.itemgetter(1), reverse=True)
noun_dict_sd_m = sorted(nouns_sd_m.items(), key=operator.itemgetter(1), reverse=True)
noun_dict_sd_pp = sorted(nouns_sd_m.items(), key=operator.itemgetter(1), reverse=True)


In [95]:
print(noun_dict_sd[0:87]) #Top nouns >= occurrences -> Top 87 nouns
count = 0
for i in range(len(noun_dict_sd[0:87])):
    count += noun_dict_sd[i][1]
print(count)

[('time', 123), ('kind', 106), ('school', 98), ('student', 96), ('today', 90), ('day', 80), ('fun', 73), ('weather', 70), ('name', 67), ('work', 66), ('hobbies', 64), ('things', 56), ('sports', 55), ('music', 53), ('job', 51), ('area', 49), ('part', 48), ('plans', 47), ('weekend', 41), ('college', 39), ('lot', 39), ('year', 38), ('family', 37), ('movies', 37), ('kids', 36), ('food', 34), ('type', 30), ('summer', 29), ('country', 29), ('place', 28), ('life', 28), ('games', 26), ('friends', 25), ('thing', 25), ('living', 25), ('state', 24), ('studies', 23), ('fan', 23), ('years', 23), ('snow', 23), ('money', 22), ('interests', 22), ('mturk', 22), ('places', 21), ('pets', 21), ('stuff', 21), ('children', 21), ('city', 20), ('major', 19), ('books', 19), ('field', 19), ('way', 19), ('winter', 17), ('movie', 17), ('person', 17), ('home', 16), ('study', 16), ('people', 16), ('business', 16), ('shows', 16), ('team', 15), ('research', 15), ('kinds', 15), ('morning', 15), ('forums', 15), ('progr

In [45]:
print(q_1)

['How are you today?', 'how are you?', 'Do you celebrate Christmas?', 'especially soups.. do you like to cook?', 'Are you from aroudn here or will be heading home for the holidays?.', 'What is your favorite foods?', 'Where do you live?', 'Are you a student here at harvard?', 'i live in Arlington.. do you travel much?', 'How about you?', 'NYC is fun, as well as p-town, what brings youi to those places?.', 'Have yoiu been to the Tennement Museum in NYC?', 'Are you an artist?', 'Have you seen the new shoe there?', 'who/what is it?', 'While I am not an artist, I went to MassArt for undgrad for graphic design and made some art there...did you go to art school?', 'the Black Mountian College show which I thought would be good.. Have you been to MassMoca?.', 'Do you do installations with your video work?', 'What do you do by day?', 'where did you go to college?.', 'what do you do now?', 'which do you prefer?', 'I play guitar, draw, and run my mouth on a lot of topics.. and u?', "didn't see tho

In [196]:
#for each noun in dict, for each question in list is noun in question? add to dict list
def sort_question_by_noun(noun_dict): #noun_dict_sd[0:100]
    questions_per_noun = {}
    for i in range(len(noun_dict)):
        noun = noun_dict_sd[i][0]
        temp_q_list = []
        for question in q_1:
            tokens = word_tokenize(question)
            for token in tokens:
                if token == noun:
                    temp_q_list.append(question)
        questions_per_noun[noun] = temp_q_list
    return questions_per_noun

In [198]:
questions_per_noun = sort_question_by_noun(noun_dict_sd[0:87])

In [200]:
#print(noun_dict_sd[0:100])
#Look at every question that contain a certain noun.
key = noun_dict_sd[86][0]

for i in range(len(questions_per_noun[key])):
    print(questions_per_noun[key][i])

I love the Fall season around here and am enjoying some outdoor activites like apple picking and hiking.. How are your courses going?
I'm on the track team here but due to injury, I didn't use all of my season of eligibility so I'll finish up using them in a 5th yr while getting a masters.. What do you study?
So what are you desiring most now from this season of your life and your last semester at Harvard?.
are you excited for the holiday season?
What season are you on, or is this one of your multiple times watching the series again?.
What did you think of the ending of season 2?
Is it your favorite season?
I am so ready.. What's your favorite season?


In [246]:
ev_q = [1, 2, 3, 4, 5]
const = len(ev_q)
for i in range(0, const):
    for j in range(i+1, const):
        print(ev_q[i],ev_q[j], i, j)
        

1 2 0 1
1 3 0 2
1 4 0 3
1 5 0 4
2 3 1 2
2 4 1 3
2 5 1 4
3 4 2 3
3 5 2 4
4 5 3 4


In [256]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
ev_q = []
ev_q= """What kind of music do you like to listen to?
What kind of movies do you like?
Whats your favorite show?
What is your favorite sci-fi show?
what is your favorite tv show?
Do you have any favorite movies or tv shows?
what do you like to watch on tv or the movies?
what do you do when you're tv bingeing?
Have you ever played basketball?
Do you watch basketball at all?
Do you like basketball?
do you watch college basketball as well?
Did you watch the Texas basketball game lastnight?"""
ev_q = ev_q.splitlines()
    
const = len(ev_q)
for n in range(0, const):
    for j in range(n+1, const):
        print(ev_q[n],ev_q[j])
        X = ev_q[n]
        Y = ev_q[j]

        # tokenization 
        X_list = word_tokenize(X)  
        Y_list = word_tokenize(Y) 

        # sw contains the list of stopwords 
        sw = stopwords.words('english')  
        l1 =[];l2 =[] 

        # remove stop words from string 
        X_set = {w for w in X_list }#if not w in sw}  
        Y_set = {w for w in Y_list }#if not w in sw} 

        # form a set containing keywords of both strings  
        rvector = X_set.union(Y_set)  
        for w in rvector: 
            if w in X_set: l1.append(1) # create a vector 
            else: l1.append(0) 
            if w in Y_set: l2.append(1) 
            else: l2.append(0) 
        c = 0

        # cosine formula  
        for i in range(len(rvector)): 
                c+= l1[i]*l2[i] 
        cosine = c / float((sum(l1)*sum(l2))**0.5) 
        print("similarity: ", cosine)
#print(ev_q)

What kind of music do you like to listen to? What kind of movies do you like?
similarity:  0.7826237921249264
What kind of music do you like to listen to? Whats your favorite show?
similarity:  0.1414213562373095
What kind of music do you like to listen to? What is your favorite sci-fi show?
similarity:  0.23904572186687872
What kind of music do you like to listen to? what is your favorite tv show?
similarity:  0.11952286093343936
What kind of music do you like to listen to? Do you have any favorite movies or tv shows?
similarity:  0.2
What kind of music do you like to listen to? what do you like to watch on tv or the movies?
similarity:  0.45643546458763845
What kind of music do you like to listen to? what do you do when you're tv bingeing?
similarity:  0.33541019662496846
What kind of music do you like to listen to? Have you ever played basketball?
similarity:  0.2581988897471611
What kind of music do you like to listen to? Do you watch basketball at all?
similarity:  0.2390457218668

In [ ]:
# Program to measure similarity between  
# two sentences using cosine similarity. 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
# X = input("Enter first string: ").lower() 
# Y = input("Enter second string: ").lower() 
X ="I love horror movies"
Y ="Lights out is a horror movie"
  
# tokenization 
X_list = word_tokenize(X)  
Y_list = word_tokenize(Y) 
  
# sw contains the list of stopwords 
sw = stopwords.words('english')  
l1 =[];l2 =[] 
  
# remove stop words from string 
X_set = {w for w in X_list if not w in sw}  
Y_set = {w for w in Y_list if not w in sw} 
  
# form a set containing keywords of both strings  
rvector = X_set.union(Y_set)  
for w in rvector: 
    if w in X_set: l1.append(1) # create a vector 
    else: l1.append(0) 
    if w in Y_set: l2.append(1) 
    else: l2.append(0) 
c = 0
  
# cosine formula  
for i in range(len(rvector)): 
        c+= l1[i]*l2[i] 
cosine = c / float((sum(l1)*sum(l2))**0.5) 
print("similarity: ", cosine) 